In [ ]:
//Web scraper for NFT Round 2 data
open System
open System.Collections.Generic
open System.Net.Http
open System.Text
open System.Text.Json
open System.Threading.Tasks
open System.IO

let nftRound2Data = List<_>()

module WebScraper =

    let Scraper (key) : Task =
        async {
            //get the url of the POST request, specifying which data we want scraped
            let url = "https://cnft.tools/toolsapi/v3/project/teddybearclub2"
            //make HTTP requests
            use client = new HttpClient()

            // Headers - mimics a request from  web browser, so its less obvious that its a scraper
            client.DefaultRequestHeaders.Add("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)")
            
            // Base payload - copy the paylod from the Networks tab - F# record
            let payloadTemplate = 
                {|
                    project = "none"
                    sort = "asc"
                    method = "rarity"
                    page = 1
                    priceOnly = "all"
                    filters = {| |}
                    sliders = 
                        {| 
                            minPrice = 0
                            maxPrice = 0
                            minRank = 0
                            maxRank = 0 
                        |}
                    instantSale = false
                    walletCheck = false
                    stakes = []
                    pageSize = 50
                |}

            let totalPages = 41
            let delayInMilliseconds = 1000

            for page in 1 .. totalPages do
                printf "Fetching page %d..." page
                // Update the page number in the payload - for pagination
                //page increments automatically
                let payload = 
                    {| payloadTemplate with 
                        page = page
                    |}

                //from F# record serialize to JSON and then 
                let jsonPayload = JsonSerializer.Serialize(payload)
                //new HTTP content object saying we are sending json
                let content = new StringContent(jsonPayload, Encoding.UTF8, "application/json")

                try
                    // Send the POST request, async F# syntax 
                    let! response = client.PostAsync(url, content) |> Async.AwaitTask
                    if response.IsSuccessStatusCode then
                        let! responseString = response.Content.ReadAsStringAsync() |> Async.AwaitTask

                        let jsonResponse = JsonSerializer.Deserialize<Dictionary<string, obj>>(responseString)

                        // :? casts result to JsonElement
                        match jsonResponse.TryGetValue(key) with
                        | true, results -> 
                            let resultsJson = results :?> JsonElement
                            if resultsJson.ValueKind = JsonValueKind.Array then
                                for element in resultsJson.EnumerateArray() do
                                    nftRound2Data.Add(element) // Add each element individually
                            else
                                printfn $"Unexpected data format for {key}"
                        | _ -> ()

                    else
                        printfn "Failed to fetch page %d. Status code: %A" page response.StatusCode
                        return()
                with
                    | ex -> 
                        printfn "Error on page %d: %s" page ex.Message
                        return()

                do! Async.Sleep(delayInMilliseconds)

            printf "Data scraping complete" 
        } |> Async.StartAsTask :> Task

WebScraper.Scraper("stats")

In [ ]:
//Will take a bit of time for the 10 elements to fill up - around 20 secs
let firstTen = nftRound2Data |> Seq.take 10 |> Seq.toList
printfn "%A" firstTen

To calculate the points for each trait, we first divide the number of occurrences of that trait by the total number of NFTs in the round, which gives us the percentage of NFTs that have that trait. We then subtract this percentage from 100, creating a direct relationship between rarity and points so rarer traits get higher points. Finally, we sum the points for each trait across different attributes (such as Background, Bear, Clothes, etc.) to calculate the final rarity score for each NFT.

In [115]:
//Getting the information regarding trait counts
open System.Text.Json

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
    TraitCount: Map<string, int>
}

let loadNFTs<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)
    
let nftRound1Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT1Traits.json"
let nftRound2Data = loadNFTs<NFTTraits list> "./JsonScraper/NFT2Traits.json"
let nftRound1Traits = List.head nftRound1Data
let nftRound2Traits = List.head nftRound2Data


In [ ]:
//Score NFT Round 1 and 2 Traits
open FSharp.Reflection
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

let processNFTTraits (nftTraits: NFTTraits) totalNfts =
    // Use FSharp.Reflection to get all the fields in the record dynamically
    let recordType = typeof<NFTTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty


let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

In [ ]:
//Displaying the total NFTs
type nftTotals = { 
    Round1: Map<string, int> 
    Round2: Map<string, int> 
    }

let nftTotalsList = [
    { Round1 = Map.ofList [("Total", 804)]; 
    Round2 = Map.ofList [("Total", 2034)] }
]

nftTotalsList.[0]


{ Round1 = map [("Total", 804)]\n Round2 = map [("Total", 2034)] } Round1 key value Total 804 Round2 key value Total 2034

In [165]:
printf "%s:" "Background"
r1ScoredTraits["Background"]

Background:

key,value
Blue Sky,80.97
Library,80.47
Mountains,81.22
Pink Sky,84.7
Starry Mountains,87.19
Starry Sky,85.45


In [164]:
printf "%s:" "Bear"
r1ScoredTraits["Bear"]

Bear:

key,value
Black Bear,89.3
Gold Bear,99.63
Panda Bear,86.44
Pink Bear,83.46
Polar Bear,80.6
Red Panda,82.71
Teddy Bear,77.86


In [163]:

printf "%s:" "Clothes"
r1ScoredTraits["Clothes"]


Clothes:

key,value
Angel Dress,98.63
Archaeologist T-shirt,98.76
Banana Suit,99.88
Basketball Jersey,99.13
Beach Tank Top,98.38
Black Denim Jacket,98.26
Black Official Suit,98.51
Black Official Vest,99
Blue Baseball Shirt,98.13
Blue Dino,98.63


In [162]:
printf "%s:" "Face"
r1ScoredTraits["Face"]

Face:

key,value
Confident,93.03
Dead,94.4
Emotionless,96.52
Expressionless,91.92
Fake Mustache,93.66
Flirty,93.28
Happy,91.42
In Love,93.16
Kissing,95.52
Mad,93.78


In [161]:
printf "%s:" "Handheld"
r1ScoredTraits["Handheld"]

Handheld:

key,value
Black Devil Trident,98.38
Broom,96.64
Clown Lollipop,97.64
Fishing Rod,98.38
Graffiti Spray,96.52
King Staff,99.5
Ninja Star,97.89
None,41.04
Pirate sword,96.89
Red Devil Trident,97.51


In [160]:

printf "%s:" "Head"
r1ScoredTraits["Head"]


Head:

key,value
Archaeologist Hat,98.76
Arrow,98.01
Black Mask,98.63
Black Red Hat,97.64
Black Round Hat,98.38
Black Square Hat,97.64
Blue Dino Hood,98.38
Blue Panama,98.26
Bubbles,97.51
Chef Hat,98.13


In [159]:
printf "%s:" "Skins"
r1ScoredTraits["Skins"]

Skins:

key,value
None,3.61
Raggedy Bear,98.01
Zombie,98.38


In [158]:
printf "%s:" "Trait Count"
r1ScoredTraits["TraitCount"]

Trait Count:

key,value
4,93.91
5,46.14
6,62.81
7,97.14


In [ ]:
//Compute the ranks of the NFTs

//Serialize JSON
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
    [<JsonPropertyName("Trait Count")>]
    TraitCount: string
}

let nftRound1NFTS = loadNFTs<NFT list> "./NFTR1.json"
let nftRound2NFTS = loadNFTs<NFT list> "./NFTR2.json"

//Obtain Ranks
let computeNFTRank nftList nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
                ("TraitCount", nft.TraitCount)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

        
let test = computeNFTRank nftRound1NFTS r1ScoredTraits
test |> List.iter(fun row -> printf "%A\n" row)


In [ ]:
//Make CSV file mapping Subject -> Rank
open System.IO

let listToCsv (data: (int * string * string * float) list) (policyId: string) (filePath: string) =
    let isNewFile = not (File.Exists(filePath))
    
    let header = "Subject,Rank"
    let rows = 
        data
        |> List.map (fun (rank, assetName, _, _) ->  
            let subject = sprintf "%s%s" policyId assetName  
            sprintf "%s,%d" subject rank  
        )
    
    if isNewFile then
        File.AppendAllLines(filePath, [header])

    let fullCsv = rows
    File.AppendAllLines(filePath, fullCsv)

let rankedNFTround1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTround2 = computeNFTRank nftRound2NFTS r2ScoredTraits
listToCsv rankedNFTround1 "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87" "NFT_Ranks.csv"
listToCsv rankedNFTround2 "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c" "NFT_Ranks.csv"

In [ ]:
//Code to calculate rewards for each subject

In [76]:
//Code to calculate Marketcap Adjustment to 8,000,000 supply

let sumValues (rewardsList: (string * float) list) =
    rewardsList |> List.sumBy (fun (_, value) -> value)

let marketcapAdjustment rewards = 
    let finalReward = (rewards*8000000)/10000000
    finalReward
